In [28]:
import pandas as pd
import numpy as np
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD


# Đọc dữ liệu từ file CSV
file_path = '../data/lyric.csv'
df = pd.read_csv(file_path)

# Xác định cột chứa lời bài hát
lyrics_column = 'lyric'  # Thay bằng tên cột cụ thể nếu khác

# Load the Vietnamese stopwords
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

def clean_text(text):
    # 1. Loại bỏ ký tự đặc biệt
    text = re.sub(r'[^\w\s]', ' ', text)  # Thay thế ký tự đặc biệt bằng khoảng trắng
    text = re.sub(r'\d+', ' ', text)  # Loại bỏ chữ số
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa

    # 2. Chuyển về chữ thường
    text = text.lower()

    # 3. Loại bỏ stopword
    if isinstance(text, str):  # Ensure text is a string
        words = text.split()
        cleaned_text = ' '.join([word for word in words if word not in stop_words])
        return cleaned_text
    return ' '.join(words)

# Áp dụng hàm xử lý lên cột chứa lời bài hát
df[lyrics_column] = df[lyrics_column].astype(str).apply(clean_text)
df.columns = ['index','name_song','lyric','label']
df.drop(['index'], axis=1,inplace=True)

processed_path = '../data/processed_data.csv'
df.to_csv(processed_path, index = False)

# Chuyển văn bản thành vector TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Bạn có thể điều chỉnh số lượng đặc trưng
X_tfidf = tfidf_vectorizer.fit_transform(df['lyric'])

# Lưu TF-IDF vectorizer
tfidf_vectorizer_path = '../model/tfidf_vectorizer.pkl'
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
print(f"TF-IDF Vectorizer đã được lưu tại: {tfidf_vectorizer_path}")

# In thông tin TF-IDF
print("Shape of TF-IDF matrix:", X_tfidf.shape)

# Giảm chiều dữ liệu bằng LSA (SVD)
num_topics = 100  # Số lượng chủ đề bạn muốn giữ lại
lsa = TruncatedSVD(n_components=num_topics, random_state=42)
X_lsa = lsa.fit_transform(X_tfidf)

# Lưu mô hình LDA
lsa_model_path = '../model/lda_model.pkl'
joblib.dump(lsa, lsa_model_path)
print(f"LDA Model đã được lưu tại: {lsa_model_path}")

# In thông tin về các thành phần chính
print("Shape of LSA-transformed matrix:", X_lsa.shape)
print("Explained variance ratio:", lsa.explained_variance_ratio_.sum())

# Lưu TF-IDF và LSA vào DataFrame
df_tfidf_lsa = pd.DataFrame(X_lsa, columns=[f"Topic{i+1}" for i in range(num_topics)])
df_tfidf_lsa['label'] = df['label']

# Lưu dữ liệu đã xử lý vào file CSV
output_path = '../data/tfidf_lsa_lyric.csv'
df_tfidf_lsa.to_csv(output_path, index=False)
print(f"Dữ liệu TF-IDF và LSA đã được lưu tại: {output_path}")

TF-IDF Vectorizer đã được lưu tại: ../model/tfidf_vectorizer.pkl
Shape of TF-IDF matrix: (978, 5000)
LDA Model đã được lưu tại: ../model/lda_model.pkl
Shape of LSA-transformed matrix: (978, 100)
Explained variance ratio: 0.40702860218196746
Dữ liệu TF-IDF và LSA đã được lưu tại: ../data/tfidf_lsa_lyric.csv


In [6]:
# Tính toán TF-IDF (Thủ công không dùng sklearn)
def compute_tf_idf(corpus):
    # Tokenize lời bài hát thành các từ riêng lẻ
    tokenized_corpus = [doc.split() for doc in corpus]

    # Tính TF (Term Frequency)
    term_frequencies = []
    for tokens in tokenized_corpus:
        tf = {}
        for word in tokens:
            tf[word] = tf.get(word, 0) + 1
        term_frequencies.append(tf)

    # Tính DF (Document Frequency)
    df = {}
    for tf in term_frequencies:
        for word in tf.keys():
            df[word] = df.get(word, 0) + 1

    # Tính IDF (Inverse Document Frequency)
    total_docs = len(corpus)
    idf = {word: np.log(total_docs / df[word]) for word in df}

    # Tính TF-IDF
    tf_idf = []
    for tf in term_frequencies:
        tf_idf_doc = {word: tf[word] * idf[word] for word in tf}
        tf_idf.append(tf_idf_doc)

    return tf_idf, idf

# Tạo ma trận TF-IDF
tf_idf_values, idf = compute_tf_idf(df[lyrics_column].astype(str).tolist())

# Chuyển đổi TF-IDF thành ma trận dạng số
unique_words = list(idf.keys())
tf_idf_matrix = []
for tf_idf_doc in tf_idf_values:
    row = [tf_idf_doc.get(word, 0) for word in unique_words]
    tf_idf_matrix.append(row)

tf_idf_matrix = np.array(tf_idf_matrix)

# Áp dụng LDA thủ công để trích xuất chủ đề
def lda(matrix, K, iterations=100, alpha=0.1, beta=0.01):
    M, V = matrix.shape  # Số tài liệu và số từ
    # Khởi tạo ngẫu nhiên chủ đề cho mỗi từ trong mỗi tài liệu
    z = np.random.randint(0, K, size=(M, V))
    
    # Đếm số lượng từ trong các chủ đề và tài liệu
    ndk = np.zeros((M, K))  # Số từ trong mỗi tài liệu thuộc mỗi chủ đề
    nkw = np.zeros((K, V))  # Số từ trong mỗi chủ đề
    nk = np.zeros(K)        # Tổng số từ trong mỗi chủ đề

    for m in range(M):
        for v in range(V):
            topic = z[m, v]
            ndk[m, topic] += matrix[m, v]
            nkw[topic, v] += matrix[m, v]
            nk[topic] += matrix[m, v]

    for it in range(iterations):
        for m in range(M):
            for v in range(V):
                if matrix[m, v] == 0:
                    continue

                topic = z[m, v]
                ndk[m, topic] -= matrix[m, v]
                nkw[topic, v] -= matrix[m, v]
                nk[topic] -= matrix[m, v]

                # Tính xác suất của mỗi chủ đề
                p_z = (ndk[m, :] + alpha) * (nkw[:, v] + beta) / (nk + V * beta)
                p_z /= p_z.sum()

                # Lấy chủ đề mới theo phân phối p_z
                new_topic = np.random.choice(K, p=p_z)
                z[m, v] = new_topic

                ndk[m, new_topic] += matrix[m, v]
                nkw[new_topic, v] += matrix[m, v]
                nk[new_topic] += matrix[m, v]

    return ndk, nkw

K = 10  # Số chủ đề
ndk, nkw = lda(tf_idf_matrix, K)

# Tạo DataFrame với các chủ đề cho mỗi tài liệu
topic_columns = [f'topic_{i+1}' for i in range(K)]
df_topics = pd.DataFrame(ndk, columns=topic_columns)

In [12]:
print(df)


                                             name_song  \
0    Nắng Có Còn Xuân & Xuân Đẹp Làm Sao & ...   
1                       Hoa Cỏ Mùa Xuân & Khá Khen   
2                    Đón Xuân & 60 Năm Cuộc Đời   
3                  Gái Xuân & Tình Yêu Màu Nắng   
4                                 Cầu Vừa Đủ Xài   
..                                                 ...   
973                                           treasure   
974                           không sao mà, em đây rồi   
975                                            ngỏ lời   
976                                           tình đầu   
977                      xe anh đến đâu em theo đến đó   

                                                 lyric       label  
0    mùa xuân ta mùa xuân hát kia trời đồi núi xanh...         vui  
1    cỏ non mềm mùa xuân hiền hoa thơm giọt sương t...         vui  
2    xuân gieo rắc ngàn hồn hoa đời vui bình minh m...         vui  
3    gái xuân trắng thân lấ

In [28]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Đọc dữ liệu từ file CSV
file_path = '../data/tfidf_lsa_lyric.csv'  # File chứa dữ liệu đầu vào

try:
    df_tfidf_lsa = pd.read_csv(file_path)
except FileNotFoundError:
    print("File không tồn tại. Vui lòng kiểm tra đường dẫn.")
    exit()

df_tfidf_lsa['label'] = df['label']

# Kiểm tra xem dữ liệu có cột nhãn không
if 'label' not in df_tfidf_lsa.columns:
    print("Dữ liệu không chứa cột nhãn ('label'). Vui lòng thêm cột nhãn để phân loại.")
    exit()

# Xử lý giá trị NaN
df_tfidf_lsa = df_tfidf_lsa.dropna()


# Tách dữ liệu thành features và labels
X = df_tfidf_lsa.drop(columns=['label'])
y = df_tfidf_lsa['label']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Thiết lập tham số cho GridSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2', None]
}

# Khởi tạo mô hình RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Thiết lập GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)

# Huấn luyện mô hình grid search
grid_search.fit(X_train, y_train)

# In kết quả tốt nhất
print("Best parameters found:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)

# Dự đoán trên tập kiểm tra với mô hình tốt nhất
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Lưu mô hình tốt nhất nếu cần thiết
import joblib
model_path = '../model/best_randomforest_model.pkl'
joblib.dump(best_rf, model_path)
print(f"Mô hình tốt nhất đã được lưu tại: {model_path}")


Fitting 3 folds for each of 648 candidates, totalling 1944 fits


C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best parameters found: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best accuracy score: 0.5768229166666666
Test Accuracy: 0.6145833333333334
Classification Report:
              precision    recall  f1-score   support

        buồn       0.64      0.73      0.68        77
   nhớ nhung       0.00      0.00      0.00         9
         vui       0.69      0.69      0.69        52
  yêu thương       0.49      0.48      0.49        54

    accuracy                           0.61       192
   macro avg       0.46      0.48      0.47       192
weighted avg       0.58      0.61      0.60       192

Mô hình tốt nhất đã được lưu tại: best_randomforest_model.pkl


C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

data = pd.read_csv('../data/tfidf_lsa_lyric.csv')
data = data.dropna()
# Tách dữ liệu thành features và labels
X = data.drop(columns=['label'])
y = data['label']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  random_state=45)

# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(bootstrap= True, max_depth= None, min_samples_leaf= 1, min_samples_split= 10, n_estimators= 100)
rf_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = rf_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Lưu mô hình nếu cần thiết
import joblib
model_path = '../model/random_forest_model.pkl'
joblib.dump(rf_model, model_path)
print(f"Mô hình đã được lưu tại: {model_path}")


Accuracy: 0.6302083333333334
Classification Report:
              precision    recall  f1-score   support

        buồn       0.59      0.75      0.66        68
         vui       0.68      0.75      0.71        51
  yêu thương       0.65      0.44      0.52        73

    accuracy                           0.63       192
   macro avg       0.64      0.64      0.63       192
weighted avg       0.64      0.63      0.62       192

Mô hình đã được lưu tại: ../model/random_forest_model.pkl


In [26]:
import joblib
import pandas as pd


model_path = '../model/random_forest_model.pkl'
best_rf = joblib.load(model_path)
print("Mô hình đã được tải thành công.")

    
df_test = pd.DataFrame([[1001,'Buồn','''yêu yêu yêu''']], columns=['index','name_song', 'lyric'])

df_test['lyric'] = df_test['lyric'].astype(str).apply(clean_text)
df_test.columns = ['index','name_song','lyric']
df_test.head()


# Chuyển văn bản thành vector TF-IDF
tfidf_vectorizer = joblib.load('../model/tfidf_vectorizer.pkl')

new_tfidf = tfidf_vectorizer.transform(df_test['lyric'])

# In thông tin TF-IDF
print("Shape of TF-IDF matrix:", new_tfidf.shape)

# Giảm chiều dữ liệu bằng LSA (SVD)
lda_model = joblib.load('../model/lda_model.pkl')

new_lda = lda_model.transform(new_tfidf)
print(new_lda.shape)

predicted_label = best_rf.predict(new_lda)

print(f"Cảm xúc dự đoán của bài hát :", predicted_label[0])


Mô hình đã được tải thành công.
Shape of TF-IDF matrix: (1, 5000)
(1, 100)
Cảm xúc dự đoán của bài hát : yêu thương


C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
